# Making Neural Network Model for Addition


## Data

In [1]:
import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

Using TensorFlow backend.


In [2]:
def gen_data_add(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2,43])        
        y.append(num1+num2)
    return np.array(X), np.array(y)

def gen_data_sub(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2,145])        
        y.append(num1-num2)
    return np.array(X), np.array(y)

def gen_data_mult(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2,242])        
        y.append(num1*num2)
    return np.array(X), np.array(y)

def gen_data_div(num_examples, range_start, range_end):
    X = []
    y = []
    for ex in range(num_examples):
        num1 = int(random.random()*(range_end - range_start) + range_start)
        num2 = int(random.random()*(range_end - range_start) + range_start)

        X.append([num1,num2,347])        
        y.append(num1/num2)
    return np.array(X), np.array(y)

def gen_data(num_examples, range_start, range_end):
    num_examples = int(num_examples/4)
    
    ax, ay = gen_data_add(num_examples, range_start, range_end)
    mx, my = gen_data_sub(num_examples, range_start, range_end)
    mux, muy = gen_data_mult(num_examples, range_start, range_end)
    dx, dy = gen_data_div(num_examples, range_start, range_end)
    
    X = np.concatenate((ax, mx, mux, dx))
    y = np.concatenate((ay, my, muy, dy))
    
    return X, y

In [3]:
x_train, y_train = gen_data(10, -50, 50)
x_test, y_test = gen_data(10, 50, 200)

x_train = x_train.reshape(x_train.shape[0], 1, x_train.shape[1])
x_test = x_test.reshape(x_test.shape[0], 1,  x_test.shape[1])
[a.shape for a in [x_train, y_train, x_test, y_test]]

NameError: name 'mx' is not defined

## RangedAccuracy Metric

In [ ]:
class RangedAccuracy(Metric):
    def __init__(self, arange=1, name="RangedAccuracy", **kwargs):
        super(RangedAccuracy, self).__init__(name=name, **kwargs)
        self.arange = tf.constant(arange, dtype=tf.float32)
        self.racc = self.add_weight(name='racc', initializer='zeros', dtype=tf.float32)
        self.in_range = self.add_weight(name='racc_ir', initializer='zeros', dtype=tf.float32)
        self.total = self.add_weight(name='racc_tot', initializer='zeros', dtype=tf.float32)
        
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_diff = tf.math.abs(y_true - y_pred)
        in_range = tf.math.less(tf.cast(y_diff, dtype=tf.float32), self.arange)
        trues = tf.reduce_sum(tf.cast(in_range, tf.float32))
        
        self.in_range = self.in_range + trues
        self.total = self.total + tf.cast(tf.size(in_range), dtype=tf.float32)
        self.racc = self.in_range / self.total
        
    
    def result(self):
        return self.racc

In [ ]:
Acc = RangedAccuracy()

## Create Models

### Model with 1 layer

In [ ]:
model_1layer = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(1,3)),
    tf.keras.layers.Dense(1)
])

model_1layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy']
)

model_1layer.summary()

### Model with 2 Layers

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(128, input_shape=(1,3)),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model.summary()

### Model 3 Layer


In [ ]:
model_3layer = tf.keras.models.Sequential([
  tf.keras.layers.Dense(256, input_shape=(1,3)),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model_3layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_3layer.summary()

### Model 4 Layer

In [ ]:
model_4layer = tf.keras.models.Sequential([
  tf.keras.layers.Dense(512, input_shape=(1,3)),
  tf.keras.layers.Dense(256),
  tf.keras.layers.Dense(128),
  tf.keras.layers.Dense(64),
  tf.keras.layers.Dense(1)
])
model_4layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_4layer.summary()

### Model 5 Layer

In [ ]:
model_5layer = tf.keras.models.Sequential([
    tf.keras.layers.Dense(1024, input_shape=(1,3)),
    tf.keras.layers.Dense(512),
    tf.keras.layers.Dense(256),
    tf.keras.layers.Dense(128),
    tf.keras.layers.Dense(64),
    tf.keras.layers.Dense(1)
])
model_5layer.compile(
    loss="MeanSquaredError",
    metrics=['accuracy'] #Acc not working, in testing
)

model_5layer.summary()

## Train Model

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min')

### Train 1 Layer Model

In [ ]:
history1 = model_1layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs=100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

### Train 2 Layer Model

In [ ]:
history2 = model.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

### Train 3 Layer Model

In [ ]:
history3 = model_3layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

### Train 4 Layer Model

In [ ]:
history4 = model_4layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

### Train 5 Layer Model

In [ ]:
history5 = model_5layer.fit(
    x_train, y_train,
    batch_size=10000,
    epochs= 100,
    validation_data=(x_test,y_test),
    callbacks = [es]
)

## Test Model

In [ ]:
acc_range = 0.5

### Test 1 Layer Model

In [ ]:
pred = model_1layer.predict(x_test)

In [ ]:
test_total = 0
acc_count = 0
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")
    
    if abs(yp - y) <= acc_range:
        acc_count = acc_count + 1
    
    test_total = test_total + 1
        
    print(f"\t accuracy={acc_count / test_total}")

### Test 2 Layer Model

In [ ]:
pred = model.predict(x_test)

In [ ]:
test_total = 0
acc_count = 0
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")
    
    if abs(yp - y) <= acc_range:
        acc_count = acc_count + 1
    
    test_total = test_total + 1
        
    print(f"\t accuracy={acc_count / test_total}")

### Test 3 Layer Model

In [ ]:
pred = model_3layer.predict(x_test)

In [ ]:
test_total = 0
acc_count = 0
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")
    
    if abs(yp - y) <= acc_range:
        acc_count = acc_count + 1
    
    test_total = test_total + 1
        
    print(f"\t accuracy={acc_count / test_total}")

### Test 4 Layer Model

In [ ]:
pred = model_4layer.predict(x_test)

In [ ]:
test_total = 0
acc_count = 0
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")
    
    if abs(yp - y) <= acc_range:
        acc_count = acc_count + 1
    
    test_total = test_total + 1
        
    print(f"\t accuracy={acc_count / test_total}")

### Test 5 Layer Model

In [ ]:
pred = model_5layer.predict(x_test)

In [ ]:
test_total = 0
acc_count = 0
for a in range(len(x_test)):
    x = x_test[a]
    y = y_test[a]
    yp = pred[a][0]
    print(f"input={x[0]} expected_output={[y]} prediction={yp} difference={yp - y} %-different from actual={abs(yp - y)/y}")
    
    if abs(yp - y) <= acc_range:
        acc_count = acc_count + 1
    
    test_total = test_total + 1
        
    print(f"\t accuracy={acc_count / test_total}")